In [1]:
from basic_tokenizer import BasicTokenizer
from regex_tokenizer import RegexTokenizer
import tiktoken


### Part1. test the function of BasicTokenizer and RegexTokenizer

In [2]:
# prepare data
with open('toy.txt', 'r') as f:
    toy_text = f.read()

print(len(toy_text))

503


In [7]:
# hyper-parameters:
vocab_size = 256 + 128

In [16]:
# create a model and train a BasicTokenizer
basic_tokenizer = BasicTokenizer()
basic_tokenizer.train(toy_text, vocab_size)
# for p, idx in tokenizer.merges.items():
#     print(f'{p} => {idx}\n')

In [17]:
# simple test the function of BasicTokenizer
encode = basic_tokenizer.encode
decode = basic_tokenizer.decode
print(decode(encode("come on rk")))

come on rk


In [9]:
# create a model and train a RegexTokenizer
test_text = toy_text
GPT2_SPLIT_PATTERN = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
re_tokenizer = RegexTokenizer(GPT4_SPLIT_PATTERN)
re_tokenizer.train(test_text, vocab_size, verbose=False)

In [10]:
# simple test the function of RegexTokenizer
re_encode = re_tokenizer.encode
re_decode = re_tokenizer.decode
re_decode(re_encode('hello word! this is hello from rk'))

'hello word! this is hello from rk'

### Part2. compare RegexTokenizer and GPT4 tiktoken

In [13]:
text = 'hello world!!!? (안녕하세요!) lol123 😉'
enc_gpt4 = tiktoken.get_encoding('cl100k_base')


In [19]:
ids_basic = basic_tokenizer.encode(text)
ids_regex = re_tokenizer.encode(text)
ids_gpt4 = enc_gpt4.encode(text)

text_basic = basic_tokenizer.decode(ids_basic)
text_gpt4 = enc_gpt4.decode(ids_gpt4)
text_regex = re_tokenizer.decode(ids_regex)
text_gpt4 == text_regex == text_basic

True